In [3]:
!pip install pyarrow==14.0.2
!pip -q install langchain huggingface_hub  langchain_openai lancedb openai  tiktoken  rank_bm25 pypdf
!pip install sentence-transformers
!pip install bitsandbytes
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd

from langchain.vectorstores import LanceDB
import lancedb
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

import transformers
from transformers import LlamaTokenizer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.documents.base import Document
from torch import cuda, bfloat16
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [ ]:
HF_AUTH='YOUR_HUGGINGFACE_AUTH_TOKEN'
model_id='meta-llama/Llama-2-7b-chat-hf'


In [6]:
df = pd.read_csv('master_with_embeddings_more_columns.csv')
df.head()

,id,title,chunk,chunk_embedded,authors,sources
0,379829911,Efficacy and Safety of Methylphenidate and Ato...,BACKGROUND AND OBJECTIVE: Methylphenidate (MPH...,"[-0.21812883019447327, 0.00931312795728445, 0....",['Z'],doi: 10.1007/s40268-023-00445-3.
1,379829912,Efficacy and Safety of Methylphenidate and Ato...,"analyze correlations associated with age, sex,...","[-0.24057511985301971, 0.13817532360553741, 0....",['Z'],doi: 10.1007/s40268-023-00445-3.
2,379829913,Efficacy and Safety of Methylphenidate and Ato...,"and AEs associated with age, sex, and differen...","[-0.4178210496902466, -0.026887519285082817, 0...",['Z'],doi: 10.1007/s40268-023-00445-3.
3,379829914,Efficacy and Safety of Methylphenidate and Ato...,response rates were 84.6% in the MPH-treated g...,"[-0.5015849471092224, -0.08108298480510712, 0....",['Z'],doi: 10.1007/s40268-023-00445-3.
4,379829915,Efficacy and Safety of Methylphenidate and Ato...,"children experienced AEs during MPH treatment,...","[-0.6629533171653748, -0.0032602110877633095, ...",['Z'],doi: 10.1007/s40268-023-00445-3.


In [7]:
documents=[]
for index, row in df.iterrows():
    doc = Document(page_content = row['chunk'],
                   metadata={'id': row['id'], 'title': row['title'], 'authors': row['authors'], 'sources': row['sources']})
    documents.append(doc)

In [8]:
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k =  5

In [9]:
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
device = 'cuda:0'
docs = [
    "An example document",
    "A second document as an example"
]

embed_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
db = lancedb.connect('/tmp/lancedb')
table = db.create_table("pandas_docs", data=[
    {"vector": embed_model.embed_query("Hello World"), "text": "Hello World", "id": "1"}
], mode="overwrite")


# Initialize LanceDB retriever
docsearch = LanceDB.from_documents(documents, embed_model, connection=table)
retriever_lancedb = docsearch.as_retriever(search_kwargs={"k": 5})

In [11]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_lancedb],
                                       weights=[0.8, 0.2])

# Example customer query
query = "What is used in brain cancer imaging?"


# Retrieve relevant documents/products
docs = ensemble_retriever.get_relevant_documents(query)

In [12]:
docs

[Document(page_content='BACKGROUND: Transformer-based models are gaining popularity in medical imaging and cancer imaging applications. Many recent studies have demonstrated the use of transformer-based models for brain cancer imaging applications such as diagnosis and tumor segmentation. OBJECTIVE: This study aims to review how different vision transformers (ViTs) contributed to advancing brain cancer diagnosis and tumor segmentation using brain image data. This study examines the different architectures developed for enhancing the task of brain tumor segmentation. Furthermore, it explores', metadata={'vector': [-0.04088253527879715, -0.021630659699440002, -0.007755259983241558, -0.11646073311567307, 0.0517856664955616, -0.08051209896802902, -0.0009551304392516613, 0.057568326592445374, 0.056550730019807816, -0.07276087254285812, -0.035762716084718704, -0.0316254198551178, 0.0022693444043397903, 0.048397138714790344, -0.03658081591129303, -0.006651713512837887, -0.07778967171907425, 0

In [13]:
model_config=transformers.AutoConfig.from_pretrained(
    model_id, use_auth_token=HF_AUTH
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1085: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

In [14]:
bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [15]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True, #what does it do?
    config = model_config,
    quantization_config=bitsAndBites_config,
    device_map='auto',
    token = HF_AUTH
)
model.eval()

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [16]:
tokenizer = LlamaTokenizer.from_pretrained(model_id,use_auth_token = HF_AUTH)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1900: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [17]:
#from assignment 4, we can change sth here, e.g. use other model, in general do some experiments
some_pipeline = transformers.pipeline(
    model= model, tokenizer=tokenizer,
    return_full_text = True,
    task='text-generation',
    temperature=0.01, #parameter of the model
    max_new_tokens=512,
    repetition_penalty = 1.1
)

In [18]:
##llm
llm = HuggingFacePipeline(pipeline = some_pipeline)
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    verbose=True,
    retriever=ensemble_retriever,
    chain_type_kwargs={
        "verbose": True },

)

In [19]:
rag_pipeline(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

BACKGROUND: Transformer-based models are gaining popularity in medical imaging and cancer imaging applications. Many recent studies have demonstrated the use of transformer-based models for brain cancer imaging applications such as diagnosis and tumor segmentation. OBJECTIVE: This study aims to review how different vision transformers (ViTs) contributed to advancing brain cancer diagnosis and tumor segmentation using brain image data. This study examines the different architectures developed for enhancing the task of brain tumor segmentation. Furthermore, it explores

highest number of parameters and thus needed a cluster of as many as 8 graphics processing units for model training. 

{'query': 'What is used in brain cancer imaging?',
 'result': ' According to the article, transformer-based models are being used in brain cancer imaging for tasks such as diagnosis and tumor segmentation. Specifically, the most popular data set used in the included studies was the brain tumor segmentation challenge data set, and different architectures were developed to enhance the task of brain tumor segmentation, including Shifted Window transformer-based architectures which have recently become the most popular choice of the research community.'}